In [1]:
!pip install pandas statsmodels
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller
!pip install arch
from statsmodels.tsa.stattools import adfuller, kpss
from arch.unitroot import PhillipsPerron

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 종속변수

In [2]:
df = pd.read_csv('/content/drive/MyDrive/EWHA/SEED/2024-2/SEED_24_2_논문2조/종속변수_원본.csv').dropna()
df.columns.values[0] = 'date'
df_col = ['date', '변액종신', '변액연금', '변액유니버셜']
df = df[df_col]

df.head()

,date,변액종신,변액연금,변액유니버셜
0,2013_11,0.00932,0.01268,0.01205
1,2013_12,0.00768,0.01306,0.01242
2,2014_01,0.00765,0.01171,0.01152
3,2014_02,0.00910,0.01174,0.01193
4,2014_03,0.00904,0.01320,0.01275


## 기본통계량

In [3]:
df.describe()

,변액종신,변액연금,변액유니버셜
count,110.000000,110.000000,110.000000
mean,0.008260,0.011005,0.009731
std,0.004062,0.002456,0.002361
min,0.001460,0.000000,0.003280
25%,0.005672,0.009645,0.009012
50%,0.007340,0.011205,0.010155
75%,0.008698,0.012093,0.011040
max,0.026990,0.020720,0.018480


## 차분

In [4]:
df_col = ['변액종신', '변액연금', '변액유니버셜']

for col in df_col:
    df[f'{col}_1차차분'] = (df[col]).diff()
    df[f'{col}_2차차분'] = df[f'{col}_1차차분'].diff()

df.head(10)

,date,변액종신,변액연금,변액유니버셜,변액종신_1차차분,변액종신_2차차분,변액연금_1차차분,변액연금_2차차분,변액유니버셜_1차차분,변액유니버셜_2차차분
0,2013_11,0.00932,0.01268,0.01205,NaN,NaN,NaN,NaN,NaN,NaN
1,2013_12,0.00768,0.01306,0.01242,-0.00164,NaN,0.00038,NaN,0.00037,NaN
2,2014_01,0.00765,0.01171,0.01152,-0.00003,0.00161,-0.00135,-0.00173,-0.00090,-0.00127
3,2014_02,0.00910,0.01174,0.01193,0.00145,0.00148,0.00003,0.00138,0.00041,0.00131
4,2014_03,0.00904,0.01320,0.01275,-0.00006,-0.00151,0.00146,0.00143,0.00082,0.00041
5,2014_04,0.00901,0.01256,0.01186,-0.00003,0.00003,-0.00064,-0.00210,-0.00089,-0.00171
6,2014_05,0.00746,0.01262,0.01206,-0.00155,-0.00152,0.00006,0.00070,0.00020,0.00109
7,2014_06,0.01042,0.01234,0.01205,0.00296,0.00451,-0.00028,-0.00034,-0.00001,-0.00021
8,2014_07,0.00740,0.01385,0.01289,-0.00302,-0.00598,0.00151,0.00179,0.00084,0.00085
9,2014_08,0.00885,0.01138,0.01157,0.00145,0.00447,-0.00247,-0.00398,-0.00132,-0.00216


## 단위근 검정 : ADF, PP, KSPP

In [5]:
def adf_test(data):
    adf_result = adfuller(data.dropna())
    print(f'ADF Statistic: {adf_result[0]}')
    print(f'p-value: {adf_result[1]}')
    print('Critical Values:')
    for key, value in adf_result[4].items():
        print(f'   {key}: {value}')

def pp_test(data):
    pp_result = PhillipsPerron(data.dropna())
    print('\n PP Test Results:')
    print(f'   Test Statistic: {pp_result.stat}')
    print(f'   p-value: {pp_result.pvalue}')
    print(f'   Lags Used: {pp_result.lags}')

def kpss_test(data, regression='c'):
    kpss_stat, p_value, lags, critical_values = kpss(data.dropna(), regression=regression)
    print('\n KPSS Test Results:')
    print(f'   KPSS Statistic: {kpss_stat}')
    print(f'   p-value: {p_value}')
    print(f'   Lags Used: {lags}')
    print('   Critical Values:')
    for key, value in critical_values.items():
        print(f'      {key}: {value}')

In [6]:
#변액종신
data = df['변액종신']

adf_test(data)
pp_test(data)
kpss_test(data)

ADF Statistic: -3.3099499638788874
p-value: 0.01443952713055186
Critical Values:
   1%: -3.492995948509562
   5%: -2.888954648057252
   10%: -2.58139291903223

 PP Test Results:
   Test Statistic: -9.205902660881954
   p-value: 1.9453392135234227e-15
   Lags Used: 13

 KPSS Test Results:
   KPSS Statistic: 0.2912384584627894
   p-value: 0.1
   Lags Used: 4
   Critical Values:
      10%: 0.347
      5%: 0.463
      2.5%: 0.574
      1%: 0.739


<ipython-input-5-06c07ab290e4>:17: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss_stat, p_value, lags, critical_values = kpss(data.dropna(), regression=regression)


In [7]:
#변액종신
data = df['변액종신_1차차분']

adf_test(data)
pp_test(data)
kpss_test(data)

ADF Statistic: -7.158523246984952
p-value: 3.010317373701207e-10
Critical Values:
   1%: -3.4948504603223145
   5%: -2.889758398668639
   10%: -2.5818220155325444

 PP Test Results:
   Test Statistic: -32.38433311490913
   p-value: 0.0
   Lags Used: 13

 KPSS Test Results:
   KPSS Statistic: 0.1409625899731653
   p-value: 0.1
   Lags Used: 33
   Critical Values:
      10%: 0.347
      5%: 0.463
      2.5%: 0.574
      1%: 0.739


<ipython-input-5-06c07ab290e4>:17: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss_stat, p_value, lags, critical_values = kpss(data.dropna(), regression=regression)


In [8]:
#변액종신
data = df['변액종신_2차차분']

adf_test(data)
pp_test(data)
kpss_test(data)

ADF Statistic: -7.24610801389912
p-value: 1.8305044679043447e-10
Critical Values:
   1%: -3.4989097606014496
   5%: -2.891516256916761
   10%: -2.5827604414827157

 PP Test Results:
   Test Statistic: -76.88961367417207
   p-value: 0.0
   Lags Used: 13

 KPSS Test Results:
   KPSS Statistic: 0.12377933675281284
   p-value: 0.1
   Lags Used: 26
   Critical Values:
      10%: 0.347
      5%: 0.463
      2.5%: 0.574
      1%: 0.739


<ipython-input-5-06c07ab290e4>:17: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss_stat, p_value, lags, critical_values = kpss(data.dropna(), regression=regression)


In [9]:
#변액연금
data = df['변액연금']
adf_test(data)
pp_test(data)
kpss_test(data)

ADF Statistic: -3.841430791622759
p-value: 0.0025107594269346703
Critical Values:
   1%: -3.492995948509562
   5%: -2.888954648057252
   10%: -2.58139291903223

 PP Test Results:
   Test Statistic: -9.865412268160043
   p-value: 4.1266372332565946e-17
   Lags Used: 13

 KPSS Test Results:
   KPSS Statistic: 0.4396587247625407
   p-value: 0.06006089449890488
   Lags Used: 3
   Critical Values:
      10%: 0.347
      5%: 0.463
      2.5%: 0.574
      1%: 0.739


In [10]:
#변액연금
data = df['변액연금_1차차분']
adf_test(data)
pp_test(data)
kpss_test(data)

ADF Statistic: -10.203592585818521
p-value: 5.8829370811862334e-18
Critical Values:
   1%: -3.4936021509366793
   5%: -2.8892174239808703
   10%: -2.58153320754717

 PP Test Results:
   Test Statistic: -33.093645968666536
   p-value: 0.0
   Lags Used: 13

 KPSS Test Results:
   KPSS Statistic: 0.14094393223949572
   p-value: 0.1
   Lags Used: 25
   Critical Values:
      10%: 0.347
      5%: 0.463
      2.5%: 0.574
      1%: 0.739


<ipython-input-5-06c07ab290e4>:17: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss_stat, p_value, lags, critical_values = kpss(data.dropna(), regression=regression)


In [11]:
#변액연금
data = df['변액연금_2차차분']
adf_test(data)
pp_test(data)
kpss_test(data)

ADF Statistic: -5.095383088379542
p-value: 1.4321786377330656e-05
Critical Values:
   1%: -3.5003788874873405
   5%: -2.8921519665075235
   10%: -2.5830997960069446

 PP Test Results:
   Test Statistic: -76.4278792400637
   p-value: 0.0
   Lags Used: 13

 KPSS Test Results:
   KPSS Statistic: 0.11804698184160148
   p-value: 0.1
   Lags Used: 24
   Critical Values:
      10%: 0.347
      5%: 0.463
      2.5%: 0.574
      1%: 0.739


<ipython-input-5-06c07ab290e4>:17: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss_stat, p_value, lags, critical_values = kpss(data.dropna(), regression=regression)


In [12]:
#변액유니버셜
data = df['변액유니버셜']
adf_test(data)
pp_test(data)
kpss_test(data)

ADF Statistic: -0.5911009875428997
p-value: 0.8729801868102998
Critical Values:
   1%: -3.4948504603223145
   5%: -2.889758398668639
   10%: -2.5818220155325444

 PP Test Results:
   Test Statistic: -8.411027702248672
   p-value: 2.1036755690533088e-13
   Lags Used: 13

 KPSS Test Results:
   KPSS Statistic: 1.1957325555567404
   p-value: 0.01
   Lags Used: 5
   Critical Values:
      10%: 0.347
      5%: 0.463
      2.5%: 0.574
      1%: 0.739


<ipython-input-5-06c07ab290e4>:17: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  kpss_stat, p_value, lags, critical_values = kpss(data.dropna(), regression=regression)


In [13]:
#변액유니버셜
data = df['변액유니버셜_1차차분']
adf_test(data)
pp_test(data)
kpss_test(data)

ADF Statistic: -7.8322246681072105
p-value: 6.248838395281269e-12
Critical Values:
   1%: -3.4948504603223145
   5%: -2.889758398668639
   10%: -2.5818220155325444

 PP Test Results:
   Test Statistic: -40.48747820191441
   p-value: 0.0
   Lags Used: 13

 KPSS Test Results:
   KPSS Statistic: 0.10690806877316095
   p-value: 0.1
   Lags Used: 21
   Critical Values:
      10%: 0.347
      5%: 0.463
      2.5%: 0.574
      1%: 0.739


<ipython-input-5-06c07ab290e4>:17: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss_stat, p_value, lags, critical_values = kpss(data.dropna(), regression=regression)


In [14]:
#변액유니버셜
data = df['변액유니버셜_2차차분']
adf_test(data)
pp_test(data)
kpss_test(data)

ADF Statistic: -5.435647573442008
p-value: 2.8496361683153433e-06
Critical Values:
   1%: -3.5003788874873405
   5%: -2.8921519665075235
   10%: -2.5830997960069446

 PP Test Results:
   Test Statistic: -82.86858147789683
   p-value: 0.0
   Lags Used: 13

 KPSS Test Results:
   KPSS Statistic: 0.10733211538532571
   p-value: 0.1
   Lags Used: 23
   Critical Values:
      10%: 0.347
      5%: 0.463
      2.5%: 0.574
      1%: 0.739


<ipython-input-5-06c07ab290e4>:17: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss_stat, p_value, lags, critical_values = kpss(data.dropna(), regression=regression)


In [15]:
df[['date', '변액종신_1차차분', '변액연금_1차차분', '변액유니버셜_1차차분']].to_csv('종속변수_1차.csv', index=False, encoding='utf-8-sig')

from google.colab import files
files.download('종속변수_1차.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [16]:
df[['date', '변액종신_2차차분', '변액연금_2차차분', '변액유니버셜_2차차분']].to_csv('종속변수_2차.csv', index=False, encoding='utf-8-sig')

from google.colab import files
files.download('종속변수_2차.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# 독립변수

In [17]:
df = pd.read_csv('/content/drive/MyDrive/EWHA/SEED/2024-2/SEED_24_2_논문2조/독립변수_원본.csv')

In [18]:
# 공백이나 특수문자 제거
df.columns = df.columns.str.strip()  # 양쪽 공백 제거
df.columns = df.columns.str.replace('(', '')  # 괄호 제거
df.columns = df.columns.str.replace(')', '')  # 괄호 제거

# 다시 열 이름 확인
print(df.columns)

Index(['date', 'CD91일', '실업률', '동행지수순환변동치', '총지수', 'KOSPI_종가'], dtype='object')


In [19]:
df.describe()

,CD91일,실업률,동행지수순환변동치,총지수,KOSPI_종가
count,112.000000,112.000000,112.000000,112.000000,112.000000
mean,1.683036,3.597321,99.808036,98.820429,2273.114643
std,0.670467,0.396712,1.080719,4.113586,357.359427
min,0.630000,2.600000,96.400000,93.116000,1754.640000
25%,1.380000,3.400000,99.400000,95.227250,2010.552500
50%,1.635000,3.600000,99.900000,98.817000,2123.090000
75%,1.877500,3.800000,100.700000,100.182500,2431.982500
max,4.020000,4.800000,101.400000,109.260000,3296.680000


In [20]:
df['로그동행지수순환변동치'] = np.log(df['동행지수순환변동치'])
df['로그총지수'] = np.log(df['총지수'])
df['로그KOSPI_종가'] = np.log(df['KOSPI_종가'])

In [21]:
df_col = ['실업률', '로그동행지수순환변동치', '로그총지수', '로그KOSPI_종가']

for col in df_col:
    df[f'{col}_1차차분'] = df[col].diff()
    df[f'{col}_2차차분'] = df[f'{col}_1차차분'].diff()

df['CD91일_1차차분'] = np.log(df['CD91일']).diff()
df['CD91일_2차차분'] = df['CD91일_1차차분'].diff()

df.head(10)

,date,CD91일,실업률,동행지수순환변동치,총지수,KOSPI_종가,로그동행지수순환변동치,로그총지수,로그KOSPI_종가,실업률_1차차분,실업률_2차차분,로그동행지수순환변동치_1차차분,로그동행지수순환변동치_2차차분,로그총지수_1차차분,로그총지수_2차차분,로그KOSPI_종가_1차차분,로그KOSPI_종가_2차차분,CD91일_1차차분,CD91일_2차차분
0,2013_09,2.66,3.0,99.7,93.419,1996.96,4.602166,4.537095,7.599381,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2013_10,2.66,3.1,100.0,93.134,2030.09,4.605170,4.534039,7.615835,0.1,NaN,0.003005,NaN,-0.003055,NaN,0.016454,NaN,0.000000,NaN
2,2013_11,2.65,3.1,100.0,93.116,2044.87,4.605170,4.533846,7.623089,0.0,-0.1,0.000000,-0.003005,-0.000193,0.002862,0.007254,-0.009200,-0.003766,-0.003766
3,2013_12,2.65,3.2,100.2,93.229,2011.34,4.607168,4.535059,7.606556,0.1,0.1,0.001998,0.001998,0.001213,0.001406,-0.016533,-0.023787,0.000000,0.003766
4,2014_01,2.65,3.2,100.5,93.730,1941.15,4.610158,4.540418,7.571036,0.0,-0.1,0.002990,0.000992,0.005359,0.004147,-0.035521,-0.018988,0.000000,0.000000
5,2014_02,2.65,3.6,100.5,93.980,1979.99,4.610158,4.543082,7.590847,0.4,0.4,0.000000,-0.002990,0.002664,-0.002696,0.019811,0.055332,0.000000,0.000000
6,2014_03,2.65,3.4,100.6,94.153,1985.61,4.611152,4.544921,7.593681,-0.2,-0.6,0.000995,0.000995,0.001839,-0.000825,0.002834,-0.016977,0.000000,0.000000
7,2014_04,2.65,3.6,100.4,94.214,1961.79,4.609162,4.545569,7.581613,0.2,0.4,-0.001990,-0.002985,0.000648,-0.001191,-0.012069,-0.014903,0.000000,0.000000
8,2014_05,2.65,3.5,100.0,94.369,1994.96,4.605170,4.547213,7.598379,-0.1,-0.3,-0.003992,-0.002002,0.001644,0.000996,0.016767,0.028836,0.000000,0.000000
9,2014_06,2.65,3.5,100.0,94.248,2002.21,4.605170,4.545930,7.602007,0.0,0.1,0.000000,0.003992,-0.001283,-0.002927,0.003628,-0.013139,0.000000,0.000000


In [22]:
data = df['CD91일']
adf_test(data)
pp_test(data)
kpss_test(data)

ADF Statistic: -1.4729744670560994
p-value: 0.5468356210687144
Critical Values:
   1%: -3.492995948509562
   5%: -2.888954648057252
   10%: -2.58139291903223

 PP Test Results:
   Test Statistic: -1.1695112782557486
   p-value: 0.6865718023344312
   Lags Used: 13

 KPSS Test Results:
   KPSS Statistic: 0.3732470850549474
   p-value: 0.08868660126941921
   Lags Used: 6
   Critical Values:
      10%: 0.347
      5%: 0.463
      2.5%: 0.574
      1%: 0.739


In [23]:
data = df['CD91일_1차차분']
adf_test(data)
pp_test(data)
kpss_test(data)

ADF Statistic: -3.6457366096053905
p-value: 0.004944492342406738
Critical Values:
   1%: -3.49181775886872
   5%: -2.8884437992971588
   10%: -2.5811201893779985

 PP Test Results:
   Test Statistic: -6.179244078356349
   p-value: 6.515868024006036e-08
   Lags Used: 13

 KPSS Test Results:
   KPSS Statistic: 0.46448287584218995
   p-value: 0.04966601895446173
   Lags Used: 5
   Critical Values:
      10%: 0.347
      5%: 0.463
      2.5%: 0.574
      1%: 0.739


In [24]:
data = df['CD91일_2차차분']
adf_test(data)
pp_test(data)
kpss_test(data)

ADF Statistic: -6.217653784681571
p-value: 5.317278729425538e-08
Critical Values:
   1%: -3.4968181663902103
   5%: -2.8906107514600103
   10%: -2.5822770483285953

 PP Test Results:
   Test Statistic: -22.767370872936016
   p-value: 0.0
   Lags Used: 13

 KPSS Test Results:
   KPSS Statistic: 0.06524325352916789
   p-value: 0.1
   Lags Used: 12
   Critical Values:
      10%: 0.347
      5%: 0.463
      2.5%: 0.574
      1%: 0.739


<ipython-input-5-06c07ab290e4>:17: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss_stat, p_value, lags, critical_values = kpss(data.dropna(), regression=regression)


In [25]:
data = df['실업률']
adf_test(data)
pp_test(data)
kpss_test(data)

ADF Statistic: -1.5269954179968486
p-value: 0.5201173485077262
Critical Values:
   1%: -3.4942202045135513
   5%: -2.889485291005291
   10%: -2.5816762131519275

 PP Test Results:
   Test Statistic: -3.6380132357230273
   p-value: 0.005074527012701498
   Lags Used: 13

 KPSS Test Results:
   KPSS Statistic: 0.29922803371726203
   p-value: 0.1
   Lags Used: 5
   Critical Values:
      10%: 0.347
      5%: 0.463
      2.5%: 0.574
      1%: 0.739


<ipython-input-5-06c07ab290e4>:17: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss_stat, p_value, lags, critical_values = kpss(data.dropna(), regression=regression)


In [26]:
data = df['실업률_1차차분']
adf_test(data)
pp_test(data)
kpss_test(data)

ADF Statistic: -4.457691387175977
p-value: 0.00023430934902177067
Critical Values:
   1%: -3.4942202045135513
   5%: -2.889485291005291
   10%: -2.5816762131519275

 PP Test Results:
   Test Statistic: -15.201391197989487
   p-value: 5.8229186652222385e-28
   Lags Used: 13

 KPSS Test Results:
   KPSS Statistic: 0.20053774704937496
   p-value: 0.1
   Lags Used: 6
   Critical Values:
      10%: 0.347
      5%: 0.463
      2.5%: 0.574
      1%: 0.739


<ipython-input-5-06c07ab290e4>:17: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss_stat, p_value, lags, critical_values = kpss(data.dropna(), regression=regression)


In [27]:
data = df['실업률_2차차분']
adf_test(data)
pp_test(data)
kpss_test(data)

<ipython-input-5-06c07ab290e4>:17: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss_stat, p_value, lags, critical_values = kpss(data.dropna(), regression=regression)


ADF Statistic: -5.941543724100933
p-value: 2.2548001494111044e-07
Critical Values:
   1%: -3.498198082189098
   5%: -2.891208211860468
   10%: -2.5825959973472097

 PP Test Results:
   Test Statistic: -51.32787552822633
   p-value: 0.0
   Lags Used: 13

 KPSS Test Results:
   KPSS Statistic: 0.11069708913675284
   p-value: 0.1
   Lags Used: 16
   Critical Values:
      10%: 0.347
      5%: 0.463
      2.5%: 0.574
      1%: 0.739


In [28]:
data = df['로그동행지수순환변동치']
adf_test(data)
pp_test(data)
kpss_test(data)

ADF Statistic: -2.826174471864201
p-value: 0.05463110565664192
Critical Values:
   1%: -3.4954932834550623
   5%: -2.8900369024285117
   10%: -2.5819706975209726

 PP Test Results:
   Test Statistic: -2.066242824321413
   p-value: 0.258303312811281
   Lags Used: 13

 KPSS Test Results:
   KPSS Statistic: 0.13730493055785892
   p-value: 0.1
   Lags Used: 6
   Critical Values:
      10%: 0.347
      5%: 0.463
      2.5%: 0.574
      1%: 0.739


<ipython-input-5-06c07ab290e4>:17: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss_stat, p_value, lags, critical_values = kpss(data.dropna(), regression=regression)


In [29]:
data = df['로그동행지수순환변동치_1차차분']
adf_test(data)
pp_test(data)
kpss_test(data)

ADF Statistic: -6.8589420762062865
p-value: 1.621111661542375e-09
Critical Values:
   1%: -3.4924012594942333
   5%: -2.8886968193364835
   10%: -2.5812552709190673

 PP Test Results:
   Test Statistic: -5.323724933602099
   p-value: 4.888880878003698e-06
   Lags Used: 13

 KPSS Test Results:
   KPSS Statistic: 0.054740255992727055
   p-value: 0.1
   Lags Used: 4
   Critical Values:
      10%: 0.347
      5%: 0.463
      2.5%: 0.574
      1%: 0.739


<ipython-input-5-06c07ab290e4>:17: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss_stat, p_value, lags, critical_values = kpss(data.dropna(), regression=regression)


In [30]:
data = df['로그동행지수순환변동치_2차차분']
adf_test(data)
pp_test(data)
kpss_test(data)

ADF Statistic: -7.939970836618747
p-value: 3.334167364712886e-12
Critical Values:
   1%: -3.4954932834550623
   5%: -2.8900369024285117
   10%: -2.5819706975209726

 PP Test Results:
   Test Statistic: -19.718936085331443
   p-value: 0.0
   Lags Used: 13

 KPSS Test Results:
   KPSS Statistic: 0.01436260637956982
   p-value: 0.1
   Lags Used: 0
   Critical Values:
      10%: 0.347
      5%: 0.463
      2.5%: 0.574
      1%: 0.739


<ipython-input-5-06c07ab290e4>:17: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss_stat, p_value, lags, critical_values = kpss(data.dropna(), regression=regression)


In [31]:
data = df['로그총지수']
adf_test(data)
pp_test(data)
kpss_test(data)

ADF Statistic: 1.3347637941369184
p-value: 0.996796661316077
Critical Values:
   1%: -3.497501033
   5%: -2.89090644
   10%: -2.5824349

 PP Test Results:
   Test Statistic: 2.20965490493296
   p-value: 0.9988903555831579
   Lags Used: 13

 KPSS Test Results:
   KPSS Statistic: 1.5523196128214658
   p-value: 0.01
   Lags Used: 6
   Critical Values:
      10%: 0.347
      5%: 0.463
      2.5%: 0.574
      1%: 0.739


<ipython-input-5-06c07ab290e4>:17: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  kpss_stat, p_value, lags, critical_values = kpss(data.dropna(), regression=regression)


In [32]:
data = df['로그총지수_1차차분']
adf_test(data)
pp_test(data)
kpss_test(data)

ADF Statistic: -2.104645288571259
p-value: 0.24265081499883312
Critical Values:
   1%: -3.497501033
   5%: -2.89090644
   10%: -2.5824349

 PP Test Results:
   Test Statistic: -8.160689756401094
   p-value: 9.161132126717824e-13
   Lags Used: 13

 KPSS Test Results:
   KPSS Statistic: 0.5935783277402031
   p-value: 0.023220152023617893
   Lags Used: 0
   Critical Values:
      10%: 0.347
      5%: 0.463
      2.5%: 0.574
      1%: 0.739


In [33]:
data = df['로그총지수_2차차분']
adf_test(data)
pp_test(data)
kpss_test(data)

ADF Statistic: -4.783069348772167
p-value: 5.859027567558403e-05
Critical Values:
   1%: -3.498198082189098
   5%: -2.891208211860468
   10%: -2.5825959973472097

 PP Test Results:
   Test Statistic: -24.0968741801938
   p-value: 0.0
   Lags Used: 13

 KPSS Test Results:
   KPSS Statistic: 0.16892304748174414
   p-value: 0.1
   Lags Used: 29
   Critical Values:
      10%: 0.347
      5%: 0.463
      2.5%: 0.574
      1%: 0.739


<ipython-input-5-06c07ab290e4>:17: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss_stat, p_value, lags, critical_values = kpss(data.dropna(), regression=regression)


In [34]:
data = df['로그KOSPI_종가']
adf_test(data)
pp_test(data)
kpss_test(data)

ADF Statistic: -1.7793683130007503
p-value: 0.3907517858588259
Critical Values:
   1%: -3.490683082754047
   5%: -2.8879516565798817
   10%: -2.5808574442009578

 PP Test Results:
   Test Statistic: -2.0770803934718503
   p-value: 0.2538299873759188
   Lags Used: 13

 KPSS Test Results:
   KPSS Statistic: 0.8530524608837599
   p-value: 0.01
   Lags Used: 6
   Critical Values:
      10%: 0.347
      5%: 0.463
      2.5%: 0.574
      1%: 0.739


<ipython-input-5-06c07ab290e4>:17: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  kpss_stat, p_value, lags, critical_values = kpss(data.dropna(), regression=regression)


In [35]:
data = df['로그KOSPI_종가_1차차분']
adf_test(data)
pp_test(data)
kpss_test(data)

ADF Statistic: -10.903422544960168
p-value: 1.1439362582461536e-19
Critical Values:
   1%: -3.4912451337340342
   5%: -2.8881954545454547
   10%: -2.5809876033057852

 PP Test Results:
   Test Statistic: -11.001624350403572
   p-value: 6.660018702759823e-20
   Lags Used: 13

 KPSS Test Results:
   KPSS Statistic: 0.09155857041605465
   p-value: 0.1
   Lags Used: 3
   Critical Values:
      10%: 0.347
      5%: 0.463
      2.5%: 0.574
      1%: 0.739


<ipython-input-5-06c07ab290e4>:17: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss_stat, p_value, lags, critical_values = kpss(data.dropna(), regression=regression)


In [36]:
data = df['로그KOSPI_종가_2차차분']
adf_test(data)
pp_test(data)
kpss_test(data)

ADF Statistic: -7.380106578582388
p-value: 8.51676169991582e-11
Critical Values:
   1%: -3.4954932834550623
   5%: -2.8900369024285117
   10%: -2.5819706975209726

 PP Test Results:
   Test Statistic: -37.61932419345893
   p-value: 0.0
   Lags Used: 13

 KPSS Test Results:
   KPSS Statistic: 0.16455696062436112
   p-value: 0.1
   Lags Used: 24
   Critical Values:
      10%: 0.347
      5%: 0.463
      2.5%: 0.574
      1%: 0.739


<ipython-input-5-06c07ab290e4>:17: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss_stat, p_value, lags, critical_values = kpss(data.dropna(), regression=regression)


In [37]:
df[['date', 'CD91일_2차차분', '실업률_2차차분', '로그동행지수순환변동치_2차차분', '로그총지수_2차차분', '로그KOSPI_종가_2차차분']].to_csv('독립변수_2차.csv', index=False, encoding='utf-8-sig')

from google.colab import files
files.download('독립변수_2차.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [38]:
df[['date', 'CD91일_1차차분', '실업률_1차차분', '로그동행지수순환변동치_1차차분', '로그총지수_1차차분', '로그KOSPI_종가_1차차분']].to_csv('독립변수_1차.csv', index=False, encoding='utf-8-sig')

from google.colab import files
files.download('독립변수_1차.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>